![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_BertForZeroShotClassification.ipynb)

## Import ONNX RoBertaForZeroShotClassification  models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `RoBertaForZeroShotClassification ` is only available since in `Spark NLP 5.2.4` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for zero shot classification via `RoBertaForZeroShotClassification` or `TFRoBertaForZeroShotClassification`. These models are usually under `Zero-Shot Classification` category and have `roberta` in their labels
- Reference: [TFRoBertaForZeroShotClassification](https://huggingface.co/docs/transformers/en/model_doc/roberta#transformers.TFRobertaForSequenceClassification)
- Some [example models](https://huggingface.co/models?pipeline_tag=zero-shot-classification&sort=downloads&search=roberta)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [1]:
!pip install -q --upgrade transformers[onnx]==4.52.3 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [cross-encoder/nli-roberta-base](https://huggingface.co/cross-encoder/nli-roberta-base) model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.
- In addition to the RoBERTa model, we also need to save the tokenizer. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import RobertaTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

MODEL_NAME = 'cross-encoder/nli-roberta-base'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME)
ort_model.save_pretrained(ONNX_MODEL)

tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(ONNX_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

Too many ONNX model files were found in onnx/model.onnx ,onnx/model_O1.onnx ,onnx/model_O2.onnx ,onnx/model_O3.onnx ,onnx/model_O4.onnx ,onnx/model_qint8_arm64.onnx ,onnx/model_qint8_avx512.onnx ,onnx/model_qint8_avx512_vnni.onnx ,onnx/model_quint8_avx2.onnx. specify which one to load by using the `file_name` and/or the `subfolder` arguments. Loading the file model.onnx in the subfolder onnx.


onnx/model.onnx:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

('onnx_models/cross-encoder/nli-roberta-base/tokenizer_config.json',
 'onnx_models/cross-encoder/nli-roberta-base/special_tokens_map.json',
 'onnx_models/cross-encoder/nli-roberta-base/vocab.json',
 'onnx_models/cross-encoder/nli-roberta-base/merges.txt',
 'onnx_models/cross-encoder/nli-roberta-base/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {ONNX_MODEL}

total 488660
-rw-r--r-- 1 root root       830 Jun 16 17:26 config.json
-rw-r--r-- 1 root root    456318 Jun 16 17:27 merges.txt
-rw-r--r-- 1 root root 498911192 Jun 16 17:26 model.onnx
-rw-r--r-- 1 root root       957 Jun 16 17:26 special_tokens_map.json
-rw-r--r-- 1 root root      1245 Jun 16 17:26 tokenizer_config.json
-rw-r--r-- 1 root root    999355 Jun 16 17:27 vocab.json


- We need to move `vocab.txt` and `merges.txt` from the tokenizer into an `assets` folder, which is where Spark NLP expects to find them. However, before doing that, we first need to convert Hugging Face’s `vocab.json` into a plain `vocab.txt` format, as Spark NLP does not support the JSON format.
- Additionally, we need to extract the `labels` and their corresponding `ids` from the model's config. This mapping will be saved as `labels.txt` inside the same `assets` folder.

In [4]:
import json

!mkdir -p {ONNX_MODEL}/assets && mv {ONNX_MODEL}/merges.txt {ONNX_MODEL}/assets/

with open(f"{ONNX_MODEL}/vocab.json") as f, open(f"{ONNX_MODEL}/assets/vocab.txt", "w") as out:
    out.write("\n".join(json.load(f)))

with open(f"{ONNX_MODEL}/assets/labels.txt", "w") as f:
    f.write("\n".join(ort_model.config.id2label[k] for k in sorted(ort_model.config.id2label)))

In [5]:
!ls -lR {ONNX_MODEL}

onnx_models/cross-encoder/nli-roberta-base:
total 488216
drwxr-xr-x 2 root root      4096 Jun 16 17:27 assets
-rw-r--r-- 1 root root       830 Jun 16 17:26 config.json
-rw-r--r-- 1 root root 498911192 Jun 16 17:26 model.onnx
-rw-r--r-- 1 root root       957 Jun 16 17:26 special_tokens_map.json
-rw-r--r-- 1 root root      1245 Jun 16 17:26 tokenizer_config.json
-rw-r--r-- 1 root root    999355 Jun 16 17:27 vocab.json

onnx_models/cross-encoder/nli-roberta-base/assets:
total 852
-rw-r--r-- 1 root root     32 Jun 16 17:27 labels.txt
-rw-r--r-- 1 root root 456318 Jun 16 17:27 merges.txt
-rw-r--r-- 1 root root 407064 Jun 16 17:27 vocab.txt


In [6]:
!cat {ONNX_MODEL}/assets/labels.txt

contradiction
entailment
neutral

Voila! We have our `vocab.txt` and `merges.txt` inside assets directory, along with the extracted labels saved in `labels.txt`.

## Import and Save RoBertaForZeroShotClassification in Spark NLP


- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [7]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 12.5 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [8]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `RoBertaForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [9]:
from sparknlp.annotator import RoBertaForZeroShotClassification

zero_shot_classifier = RoBertaForZeroShotClassification.loadSavedModel(
      ONNX_MODEL,
      spark
    )\
    .setInputCols(["document", "token"]) \
    .setOutputCol("class") \
    .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [10]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [11]:
!rm -rf {MODEL_NAME}

Awesome 😎  !

This is your RoBertaForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [12]:
! ls -l {MODEL_NAME}_spark_nlp_onnx

total 487308
drwxr-xr-x 5 root root      4096 Jun 16 17:30 fields
drwxr-xr-x 2 root root      4096 Jun 16 17:30 metadata
-rw-r--r-- 1 root root 498987456 Jun 16 17:30 roberta_classification_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny BertForZeroShotClassification model 😊

In [13]:
zero_shot_classifier_loaded = RoBertaForZeroShotClassification.load("./{}_spark_nlp_onnx".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [14]:
zero_shot_classifier_loaded.getClasses()

['entailment', 'neutral', 'contradiction']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [15]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = (
    DocumentAssembler()
    .setInputCol("text")
    .setOutputCol("document")
)

tokenizer = (
    Tokenizer()
    .setInputCols("document")
    .setOutputCol("token")
)

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

texts = [
    ["I have a problem with my iPhone that needs to be resolved asap!!"],
    ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
    ["I have a phone and I love it!"],
    ["I really want to visit Germany and I am planning to go there next year."],
    ["Let's watch some movies tonight! I am in the mood for a horror movie."],
    ["Have you watched the match yesterday? It was a great game!"],
    ["We need to hurry up and get to the airport. We are going to miss our flight!"]
]
input_df = spark.createDataFrame(texts, ["text"])

model = pipeline.fit(input_df)
results = model.transform(input_df)

results.select("text", "class.result").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------+------------+
|text                                                                                                          |result      |
+--------------------------------------------------------------------------------------------------------------+------------+
|I have a problem with my iPhone that needs to be resolved asap!!                                              |[weather]   |
|Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app.|[weather]   |
|I have a phone and I love it!                                                                                 |[weather]   |
|I really want to visit Germany and I am planning to go there next year.                                       |[technology]|
|Let's watch some movies tonight! I am in the mood for a horror movie.                                         |[mobil

That's it! You can now go wild and use hundreds of `RoBertaForZeroShotClassification` models from HuggingFace 🤗 in Spark NLP 🚀
